In [1]:
# UNIFICAR OS PARQUETS - VERSÃO LIMPA

import os
import pandas as pd
from glob import glob

# Caminhos
pasta_parquets = "../../banco/parquet"
saida_parquet = "../../banco/parquet_unificado/sih_rs.parquet"

# Colunas desejadas (corrigida DT_INTER)
colunas_desejadas = [
     'ESPEC', 'N_AIH', 'IDENT', 'CEP', 'MUNIC_RES', 'NASC', 'SEXO', 'DT_INTER', 'DT_SAIDA',
    'UTI_MES_TO', 'MARCA_UTI', 'UTI_INT_TO', 'DIAR_ACOM', 'QT_DIARIAS', 'PROC_SOLIC', 'PROC_REA', 'VAL_SH', 'VAL_SP', 'VAL_TOT',
    'VAL_UTI',  'NATUREZA', 'CNES',  'NAT_JUR', 'GESTAO', 'IND_VDRL', 'IDADE', 'DIAG_PRINC', 
    'DIAG_SECUN', 'COBRANCA', 'MORTE',  'MUNIC_MOV', 'DIAS_PERM', 'NACIONAL', 
    'NUM_FILHOS', 'INSTRU', 'CID_NOTIF', 'CONTRACEP1', 'CONTRACEP2', 'GESTRICO', 'INSC_PN', 'CBOR',
    'CNAER', 'VINCPREV', 'INFEHOSP', 'CID_ASSO', 'CID_MORTE', 'COMPLEX', 'RACA_COR', 'ETNIA',
    'DIAGSEC1', 'DIAGSEC2', 'DIAGSEC3', 'DIAGSEC4', 'DIAGSEC5', 'DIAGSEC6', 'DIAGSEC7', 'DIAGSEC8', 'DIAGSEC9', 
    'TPDISEC1', 'TPDISEC2', 'TPDISEC3', 'TPDISEC4', 'TPDISEC5', 'TPDISEC6', 'TPDISEC7', 'TPDISEC8', 'TPDISEC9', 'COD_IDADE'
]

# Buscar arquivos parquet recursivamente
arquivos = glob(os.path.join(pasta_parquets, "**/*.parquet"), recursive=True)

if len(arquivos) == 0:
    print("ERRO: Nenhum arquivo .parquet encontrado!")
    exit()

print(f"Encontrados {len(arquivos)} arquivos parquet")
print("Iniciando processamento...")

# Lista para armazenar os DataFrames
dfs = []
arquivos_processados = 0
arquivos_com_erro = 0

# Itera sobre os arquivos .parquet
for i, arquivo in enumerate(arquivos):
    # Mostrar progresso a cada 50 arquivos ou nos últimos 10
    if i % 50 == 0 or i >= len(arquivos) - 10:
        print(f"Processando arquivo {i+1}/{len(arquivos)}: {os.path.basename(arquivo)}")
    
    try:
        df = pd.read_parquet(arquivo)
        
        # Adiciona colunas ausentes com NaN
        for coluna in colunas_desejadas:
            if coluna not in df.columns:
                df[coluna] = pd.NA

        # Mantém apenas as colunas desejadas 
        df = df[colunas_desejadas]
        
        dfs.append(df)
        arquivos_processados += 1
        
    except Exception as e:
        print(f"ERRO no arquivo {os.path.basename(arquivo)}: {e}")
        arquivos_com_erro += 1
        continue

print(f"\nResumo do processamento:")
print(f"  Arquivos processados: {arquivos_processados}/{len(arquivos)}")
if arquivos_com_erro > 0:
    print(f"  Arquivos com erro: {arquivos_com_erro}")

if len(dfs) == 0:
    print("ERRO: Nenhum arquivo foi processado com sucesso!")
else:
    print(f"\nConcatenando {len(dfs)} DataFrames...")
    
    # Concatena em lotes para evitar sobrecarga de memória
    lote_size = 50
    df_lotes = []
    
    for i in range(0, len(dfs), lote_size):
        lote = dfs[i:i+lote_size]
        print(f"  Concatenando lote {i//lote_size + 1}/{(len(dfs)-1)//lote_size + 1}")
        df_lote = pd.concat(lote, ignore_index=True)
        df_lotes.append(df_lote)
        
        # Limpar memória
        del lote
    
    print("  Concatenação final...")
    df_final = pd.concat(df_lotes, ignore_index=True)
    
    # Limpar memória
    del dfs, df_lotes
    
    # Cria a pasta de saída, se necessário
    os.makedirs(os.path.dirname(saida_parquet), exist_ok=True)

    # Salva o arquivo unificado
    print("Salvando arquivo unificado...")
    df_final.to_parquet(saida_parquet, index=False, engine="pyarrow", compression="snappy")

    print(f"\nSUCESSO!")
    print(f"Arquivo criado: sih_rs.parquet")
    print(f"Total de registros: {len(df_final):,}")
    print(f"Tamanho: {os.path.getsize(saida_parquet) / (1024*1024):.1f} MB")

Encontrados 670 arquivos parquet
Iniciando processamento...
Processando arquivo 1/670: RDRS1807.parquet
Processando arquivo 51/670: RDRS1008.parquet
Processando arquivo 101/670: RDRS1005.parquet
Processando arquivo 151/670: RDRS1105.parquet
Processando arquivo 201/670: 1f0486fed8374f8a907e594c73dc17b1-0.parquet
Processando arquivo 251/670: 1e5d29aac9df43ea80fb23fca5b7c6ca-0.parquet
Processando arquivo 301/670: 0130edbbc51f40fc8db71e5b8c155d7f-0.parquet
Processando arquivo 351/670: 745dd286a8ff461896c8180872916cbf-0.parquet
Processando arquivo 401/670: ee4e4dd3fa254661931c712f86facc7e-0.parquet
Processando arquivo 451/670: 20050fbf622a40769081cc40095922f2-0.parquet
Processando arquivo 501/670: 8c7a3bd79bd14fdebb859e7dfa83fd9e-0.parquet
Processando arquivo 551/670: 5a186e09e1c34fa69e655c34a0cf3f95-0.parquet
Processando arquivo 601/670: 065ddfa5e29b4b37bd7545fa49275bb9-0.parquet
Processando arquivo 651/670: 78071ed9aba546d7ac5bbc5455376e26-0.parquet
Processando arquivo 661/670: e17c21fc6c